#Installing packages & connecting drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install  pyedflib
!pip install einops
!pip install gdown

!gdown "1jPkJSiTzxoo_Uxt_QLzgfvG48XWtIwEH&confirm=t"   

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 803.6 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.3 MB/s eta 0:00:00a 0:00:01
Downloading...
From: https://drive.google.com/uc?id=1jPkJSiTzxoo_Uxt_QLzgfvG48XWtIwEH&confirm=t
To: /tf/artifacts_5s_128hz.pkl
 61%|███████████████████████▎              | 3.43G/5.57G [01:47<02:11, 16.3MB/s]

#Load data 

In [13]:
import pickle

data_path = 'artifacts_5s_128hz.pkl'  # replace with your own file path
with open(data_path, 'rb') as f:
    data = pickle.load(f)

In [14]:
import tensorflow as tf
import numpy as np
import os
import einops
import shutil
import matplotlib.pyplot as plt
import pickle
import zipfile
from tqdm import tqdm
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def to_pickle(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'Pickled: {os.path.getsize(filename) / 1e6} MB / {os.path.getsize(filename) / 1e9} GB')

def to_zip(pickle_filename, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(pickle_filename)
    print(f'Zipped: {os.path.getsize(zip_filename) / 1e6} MB / {os.path.getsize(zip_filename) / 1e9} GB')

def from_zip(zip_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        zipf.extractall()
    print(f'Unzipped: {os.path.getsize(zip_filename) / 1e6} MB / {os.path.getsize(zip_filename) / 1e9} GB')
        
def from_pickle(pickle_filename):
    with open(pickle_filename, 'rb') as f:
        arr = pickle.load(f)
        print(f'Loaded: {arr.shape}')
        return arr

def pickle_check(pickle_filename):
    print('')
    print('Checking pickle and zip functions...')
    os.system(f'cp {pickle_filename} test.pkl')
    data = from_pickle('test.pkl')
    to_zip('test.pkl', 'test.zip')
    from_zip('test.zip')
    loaded_data = from_pickle('test.pkl')
    assert np.array_equal(data, loaded_data)
    os.remove('test.zip')
    os.remove('test.pkl')
    print('Success!\n')

def split_files(folder_path, num_subfolders):
    # Get a list of all files in the folder
    file_list = os.listdir(folder_path)
    num_files = len(file_list)

    # Calculate the number of files to put in each subfolder
    files_per_subfolder = int(num_files / num_subfolders)
    
    # Create the subfolders if they don't already exist
    subfolder_names = [f"{folder_path}/{i}of{num_subfolders}" for i in range(1, num_subfolders + 1)]
    for subfolder_name in subfolder_names:
        os.makedirs(subfolder_name, exist_ok=True)
    
    # Copy the files into the subfolders
    progress_bar = tqdm(total=num_files)
    subfolder_idx = 0
    num_copied = 0
    for file_name in file_list:
        source_path = os.path.join(folder_path, file_name)
        destination_folder = subfolder_names[subfolder_idx]
        destination_path = os.path.join(destination_folder, file_name)
        shutil.move(source_path, destination_path)
        num_copied += 1
        progress_bar.update(1)
        if num_copied == files_per_subfolder:
            num_files_in_subfolder = len(os.listdir(destination_folder))
            print(f"{destination_folder}: {num_files_in_subfolder}")
            if subfolder_idx < num_subfolders - 1:
                subfolder_idx += 1
                num_copied = 0
    
    progress_bar.close()

def load_pickled_data(root_dir = 'data/training', file_name=None, folder_name='300'):
    data = []
    if file_name is None:
        path = os.path.join(root_dir, f"{folder_name}")

        if len(os.listdir(path)[0].split('_')) > 3:
            frequency = int(os.listdir(path)[0].split('_')[2][:-2])
        else:
            frequency = 1

        with open(os.path.join(path, file_name), "rb") as f:
            data.append(pickle.load(f))
    else:
        if file_name.endswith(".pkl"):
            with open(file_name, "rb") as f:
                data.append(pickle.load(f))
        
        if len(file_name.split('_')) > 3:
            frequency = int(file_name.split('_')[2][:-2])
        else:
            frequency = 1

    data = np.vstack(data)
    print('\nLoading:\n', data.shape, '\n', 'Total hours:', round(data.shape[0] * data.shape[2] / 60 / 60 / frequency, 3), '\n')
    return data

def save_to_pickle_multiple(data, num_files=10, target_path="."):
    os.makedirs(target_path, exist_ok=True)

    samples_per_file = data.shape[0] // num_files
    remainder = data.shape[0] % num_files
    for i in tqdm(range(num_files)):
        start_idx = i * samples_per_file
        end_idx = (i + 1) * samples_per_file
        if i == num_files - 1:
            end_idx = data.shape[0]
        if i < remainder:
            end_idx += 1
        subset_data = data[start_idx:end_idx]
        with open(os.path.join(target_path, f'data_{data.shape[2]}_1hz_{i+1}.pkl'), 'wb') as f:
            pickle.dump(subset_data, f)
def plot(data, save_name=None, show=False, annotate=False, artifact=None):
        data = np.squeeze(data)

        if annotate:
            names = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'FZ', 'CZ', 'PZ']

            if artifact is not None:
                # if artifact is a list 
                if isinstance(artifact, list):
                    for i in artifact:
                        names.append(i)
                else:
                    names = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'FZ', 'CZ', 'PZ', artifact]

            if data.shape[0] == len(names):
                fig = plt.figure(figsize=(10,5), dpi=100)
                for i in range(len(names)):
                    plt.subplot(len(names), 1, i+1)
                    plt.plot(data[i, :])
                    if annotate:
                        plt.annotate(names[i], xy=(1.05, 0.5), xycoords='axes fraction', horizontalalignment='right', verticalalignment='center')
            elif data.shape[-1] == len(names):
                fig = plt.figure(figsize=(10,5), dpi=100)
                for i in range(len(names)):
                    plt.subplot(len(names), 1, i+1)
                    plt.plot(data[:, i])
                    if annotate:
                        plt.annotate(names[i], xy=(1.05, 0.5), xycoords='axes fraction', horizontalalignment='right', verticalalignment='center')

        else:
            fig = plt.figure(figsize=(10,5), dpi=100)
            for i in range(len(data)):
                plt.subplot(len(data), 1, i+1)
                plt.plot(data[i, :])
                if annotate:
                    plt.annotate(names[i], xy=(1.05, 0.5), xycoords='axes fraction', horizontalalignment='right', verticalalignment='center')


        if save_name:
            plt.savefig(save_name)
        if show:
            plt.show()

In [15]:
dataset = load_pickled_data(root_dir = 'data/training', file_name= 'artifacts_5s_128hz.pkl', folder_name='300')



Loading:
 (51841, 21, 640) 
 Total hours: 9216.178 



#Preprocessing

In [27]:
def preprocessing_transformer(dataset, batchsize, balance=False):
    
   # target is stored in last channel 

    # rearrange data so that the channels are in the last dimension (following convention)
    dataset = einops.rearrange(dataset, 'b c t -> b t c')
    x_data = dataset[:,:,:-2]
    y_data_1 = dataset[:,:,-1] 
    y_data_2 = dataset[:,:,-2]  

    if balance:

        # if we want to reduce the dataset to only the samples that contain positive examples:
        indices_1 = np.where(np.any(y_data_1 == 1, axis=1))[0]
        indices_2  = np.where(np.any(y_data_2 == 1, axis=1))[0]
        indices_comb = np.union1d(indices_1, indices_2)



        # Extract the data and labels corresponding to these indices
        reduced_data = x_data[indices_comb]
        reduced_labels_1 = y_data_1[indices_comb]
        reduced_labels_2 = y_data_2[indices_comb]  

    # create a tensorflow dataset 
    ds = tf.data.Dataset.from_tensor_slices((reduced_data, reduced_labels_1, reduced_labels_2))
    
    ds = ds.map(lambda x,y1, y2: (x, tf.cast(y1, tf.int32), tf.cast(y2, tf.int32)))
    ds = ds.shuffle(1000).batch(batchsize).cache().prefetch(tf.data.experimental.AUTOTUNE)
   
    return ds

In [8]:
 def split(data, train=0.8, val=0.1, test=0.1):
        """
        split the data into three sets namely training, validation and test
        Args:
            data: np.array
            train: float
            val: float
            test: float
        Returns: 
            A List of with the three datasets
        """

        train_split_idx = int(data.shape[0]*0.8) # get index for split between train and val
        val_split_idx = train_split_idx + int(data.shape[0]*0.1) # get index for split between val and test
        train_ds = data[:train_split_idx, :, :] # split train
        val_ds = data[train_split_idx:val_split_idx, :, :] # split val
        test_ds = data[val_split_idx:, :, :] # split test
        return train_ds, val_ds, test_ds

In [28]:

ds_train, ds_val, ds_test = split(dataset)

ds_train_post = preprocessing_transformer(ds_train,
                                    batchsize = 52, 
                                    balance = True)
ds_val_post = preprocessing_transformer(ds_val,
                                    batchsize = 52, 
                                    balance = True)
ds_test_post = preprocessing_transformer(ds_test,
                                    batchsize = 52, 
                                    balance = True)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 640, 19), dtype=tf.float64, name=None), TensorSpec(shape=(None, 640), dtype=tf.int32, name=None), TensorSpec(shape=(None, 640), dtype=tf.int32, name=None))>

In [29]:
train_dataset = ds_train_post.map(lambda x, y1, y2: (x, (y1, y2)))
x_train, (y_train1, y_train2) = next(iter(train_dataset))

val_dataset = ds_val_post.map(lambda x, y1, y2: (x, (y1, y2)))
x_val, (y_val1, y_val2) = next(iter(train_dataset))

test_dataset = ds_test_post.map(lambda x, y1, y2: (x, (y1, y2)))
x_val, (y_val1, y_val2) = next(iter(train_dataset))

2023-03-27 16:04:47.555475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [11497,640]
	 [[{{node Placeholder/_2}}]]
2023-03-27 16:04:47.555822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [11497,640,19]
	 [[{{node Placeholder/_0}}]]
2023-03-27 16:04:47.746187: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have

##Counting Balance of Dataset


In [ ]:
# How imbalanced is this dataset?
dataset = einops.rearrange(dataset, 'b c t -> b t c')
x_data = dataset[:,:,:-2]
y_data_1 = dataset[:,:,-1] 
y_data_2 = dataset[:,:,-2]  

# if we want to reduce the dataset to only the samples that contain positive examples:
indices_1 = np.where(np.any(y_data_1 == 1, axis=1))[0]
indices_2  = np.where(np.any(y_data_2 == 1, axis=1))[0]
indices_comb = np.union1d(indices_1, indices_2)

# Extract the data and labels corresponding to these indices
reduced_data = x_data[indices_comb]
reduced_labels_1 = y_data_1[indices_comb]
reduced_labels_2 = y_data_2[indices_comb]  

count_0 = 0
count_1 = 0

# Iterate over the dataset and count the number of 1s and 0s
for target_seq in reduced_labels:
    
    count_0 += tf.math.count_nonzero(target_seq == 0)
    count_1 += tf.math.count_nonzero(target_seq == 1)

# Compute the fraction of 1s in the dataset
fraction_1 = count_1 / (count_0 + count_1)
print(count_0)
# Print the fraction of 1s in the dataset
print(f"Fraction of 1s in the dataset: {fraction_1:.2f}")



# This dataset is crazy imbalanced! only 4% of labels are positive

# for artefacts 300s_100Hz its 13%
# for artefacts 300s_128Hz its 14%

#Models


##The Baseline Model


In [24]:
class TestModel(tf.keras.Model):

  def __init__(self, num_units, num_layers, num_heads, num_conv_layers, aggregation_mode):

    super(TestModel, self).__init__()
    
    self.aggregation_mode = aggregation_mode
    self.conv_block = []
    
    for i in range(num_conv_layers):
        
        self.conv_block.append(tf.keras.layers.Conv1D(8*i + 8, 3, activation='relu'))
        # max pooling throws an error!
        #self.conv_block.append(tf.keras.layers.MaxPooling2D(2))
        
    self.dense_layers =[]
    
    for i in range(num_layers):
            self.dense_layers.append(tf.keras.layers.Dense(num_units, activation='relu'))
    
    self.mha = tf.keras.layers.MultiHeadAttention(num_heads, 640)
    self.dropout = tf.keras.layers.Dropout(0.3)
    self.dense = tf.keras.layers.Dense(640, activation = "sigmoid")
    self.dense2 = tf.keras.layers.Dense(640, activation = "sigmoid")
    

  def call(self, x):
    
    for layer in self.conv_block:
        x = layer(x)
                
    x = self.mha(x, x)
    x = self.dropout(x)
    
    for layer in self.dense_layers:
        x = layer(x)
        
    output_eye = self.dense(x)
    output_muscle = self.dense2(x)

    if self.aggregation_mode == 1:
    # assuming that an artifact occurring in any pair of channels are independent events, the
    # probability of it occurring in any one is:

        output_eye =  1-tf.math.reduce_prod(output_eye, axis=1)
        output_muscle = 1- tf.math.reduce_prod(output_muscle, axis=1)
    
    else: 
        output_eye = tf.math.reduce_mean(output_eye, axis=1)
        output_muscle = tf.math.reduce_mean(output_muscle, axis=1)

    return output_eye, output_muscle




## Transformer Convolutional NN (Peh et al)

In [ ]:
import tensorflow as tf

input_shape = (32, 250,19)
# Define the CNN architecture
cnn = tf.keras.Sequential([
    # Convolutional layer 1 with 8 filters, filter size 3, and ReLU activation
    tf.keras.layers.Conv2D(8, 3, activation='relu', input_shape=input_shape),
    # Max-pooling with stride 2
    tf.keras.layers.MaxPooling2D(2),
    # Convolutional layer 2 with 16 filters, filter size 3, and ReLU activation
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    # Max-pooling with stride 2
    tf.keras.layers.MaxPooling2D(2),
    # Convolutional layer 3 with 32 filters, filter size 3, and ReLU activation
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding = 'same'),
    # Max-pooling with stride 2
    tf.keras.layers.MaxPooling2D(2),
    # Convolutional layer 4 with 64 filters, filter size 3, and ReLU activation
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding = 'same'),
    # Max-pooling with stride 2
    tf.keras.layers.MaxPooling2D(2),
    # Convolutional layer 5 with 128 filters, filter size 3, and ReLU activation
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding = 'same'),
    # Max-pooling with stride 2
    #tf.keras.layers.MaxPooling2D(2)
])

from tensorflow.keras.layers import Layer

class Transformer_encoder(tf.keras.Model):

  def __init__(self):
    super(Transformer_encoder, self).__init__()
    self.flatten = tf.keras.layers.Flatten(),
    # Add a dense layer for query, key, and value
    self.dense = tf.keras.layers.Dense(256, activation='relu'),
    # Use the Multi-Head Attention layer for the transformer
    self.mha = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim = 250),
    # Add a dense layer for the feed-forward network
    self.dense2 = tf.keras.layers.Dense(1024, activation='relu')

  def call(self, input):
    x = self.flatten(input)
    
    #print(input)
    #print(type(x))
    #x = tf.keras.layers.Flatten()(x)
    #print(x)
    #x = self.dense(x)
    x = self.mha(x,x)
    x = self.dense2(x)
    return x


transformer_encoder = Transformer_encoder()
# Combine the CNN and Transformer encoder

#cnn(tf.keras.Input(shape = (32, 250,19)))
cnn(tf.keras.Input(shape = (32, 250,19)))

# Training

##Special Callback to see Outputs and Targets

In [ ]:
class PrintOutputsCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_dataset):
        super(PrintOutputsCallback, self).__init__()
        self.test_dataset = test_dataset

    def on_epoch_end(self, epoch, logs=None):
      
        # Get a batch of test data
        x, y_true = next(iter(self.test_dataset))
        
        # Predict the outputs for this batch
        y_pred = self.model.predict(x)
        
        # Print some outputs and targets
        print(f"Outputs: {y_pred[:3]}")
        print(f"Targets: {y_true[:3]}")

In [25]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

import os

if not os.path.exists('checkpoint_dir'):
    os.makedirs('checkpoint_dir')


log_dir = "logs/fit/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_callback = ModelCheckpoint(
    filepath='checkpoint_dir/model_{epoch:02d}.h5',
    save_weights_only=True,
    save_best_only=False,
    monitor='val_loss',
    verbose=1
)


## Normal Training

In [44]:
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
hparams = {
        HP_NUM_UNITS: 640,
        HP_NUM_LAYERS: 1,
        HP_NUM_HEADS: 4,
        HP_NUM_CONVLAYERS: 0,
        HP_AGGREGATION: 0
                }

model = TestModel(TestModel(hparams[HP_NUM_UNITS], hparams[HP_NUM_LAYERS], hparams[HP_NUM_HEADS], hparams[HP_NUM_CONVLAYERS], hparams[HP_AGGREGATION])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.BinaryAccuracy()])

#model.predict(ds_train_post.take(1))

model.fit(train_dataset, epochs=1, validation_data = val_dataset, callbacks=[tensorboard_callback])

222/222 [==============================] - 16s 59ms/step - loss: 1.2326 - output_1_loss: 0.5821 - output_2_loss: 0.6505 - output_1_precision_1: 0.3564 - output_1_binary_accuracy: 0.7038 - output_2_precision_1: 0.5923 - output_2_binary_accuracy: 0.6271 - val_loss: 1.1757 - val_output_1_loss: 0.5757 - val_output_2_loss: 0.6000 - val_output_1_precision_1: 0.3860 - val_output_1_binary_accuracy: 0.6999 - val_output_2_precision_1: 0.9446 - val_output_2_binary_accuracy: 0.7280


##Hyperparameter Tuning

### Set up grid and model for gridsearch

In [57]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import time

# Define the hyperparameters you want to tune
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([256, 640]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1, 2,3,4]))
HP_NUM_HEADS = hp.HParam('num_heads', hp.Discrete([4,8]))
HP_NUM_CONVLAYERS = hp.HParam('num_convlayers', hp.Discrete([1,2,3,4]))

#can probably take this out since 1 seems markedly worse than 0 after 1 epoche
HP_AGGREGATION = hp.HParam('aggregation', hp.Discrete([0,1]))

# Define the metrics you want to record
METRIC_ACCURACY = 'accuracy'

# Define a function to build your model

def create_model(hparams):

    model = TestModel(hparams[HP_NUM_UNITS], hparams[HP_NUM_LAYERS], hparams[HP_NUM_HEADS], hparams[HP_NUM_CONVLAYERS], hparams[HP_AGGREGATION])
    return model


# Define a function to run a single experiment

def run_experiment(hparams, epochs, logdir, savedir, checkpointdir):
    
    model = create_model(hparams)
   
 
    

    # Create the model using the hyperparameters
    # Compile the model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.BinaryAccuracy()])

        # Train the model

    model.fit(train_dataset, 
              epochs=epochs, 
              validation_data = val_dataset,
              callbacks=[tf.keras.callbacks.TensorBoard(logdir),  # log metrics
                        hp.KerasCallback(logdir, hparams),  # log hparams
                        tf.keras.callbacks.ModelCheckpoint(filepath= os.path.join(checkpointdir, "ckpt_{epoch}") ,monitor='val_loss',save_weights_only=True), # save checkpoints when val loss goes down
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)], # early stopping in the case that loss doesnt go down for 3 epochs
              ) 

    

    # Evaluate the model
    accuracy = model.evaluate(test_dataset)
    model.save(savedir)
    return accuracy


### Search the grid

In [58]:
# Define the hyperparameter search space

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

session_num = 0
for num_units in HP_NUM_UNITS.domain.values:
    for num_layers in HP_NUM_LAYERS.domain.values:
       for num_heads in HP_NUM_HEADS.domain.values:
         for num_convlayers in HP_NUM_CONVLAYERS.domain.values:
            for aggregation in HP_AGGREGATION.domain.values:
                hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_NUM_LAYERS: num_layers,
                    HP_NUM_HEADS: num_heads,
                    HP_NUM_CONVLAYERS: num_convlayers,
                    HP_AGGREGATION: aggregation
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})

                # Run a single experiment
                accuracy = run_experiment(
                    hparams=hparams,
                    epochs = 1,
                    logdir='logs/hparam_tuning/' + run_name, 
                    savedir='logs/models/'+run_name, 
                    checkpointdir='logs/checkpoints'+run_name)

                # Record the results in TensorBoard
                #with tf.summary.create_file_writer('logs/hparam_tuning/' + str(session_num)).as_default():
                 #   hp.hparams_config(
                  #      hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
                   ##     metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
                     #   )
                session_num += 1

--- Starting trial: run-0
{'num_units': 256, 'num_layers': 1, 'num_heads': 4, 'num_convlayers': 1, 'aggregation': 0}
26/26 [==============================] - 1s 24ms/step - loss: 1.1971 - output_1_loss: 0.5562 - output_2_loss: 0.6408 - output_1_precision_7: 0.5556 - output_1_binary_accuracy: 0.7412 - output_2_precision_7: 0.6367 - output_2_binary_accuracy: 0.6246


2023-03-27 16:43:28.094529: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,8]
	 [[{{node inputs}}]]
2023-03-27 16:43:28.103806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,8]
	 [[{{node inputs}}]]
2023-03-27 16:43:28.109125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,256]
	 [[{{node inputs}}]]
2023-03-

--- Starting trial: run-1
{'num_units': 256, 'num_layers': 1, 'num_heads': 4, 'num_convlayers': 1, 'aggregation': 1}
26/26 [==============================] - 1s 23ms/step - loss: 20.6758 - output_1_loss: 11.2120 - output_2_loss: 9.4639 - output_1_precision_8: 0.2648 - output_1_binary_accuracy: 0.2648 - output_2_precision_8: 0.3794 - output_2_binary_accuracy: 0.3794


2023-03-27 16:43:46.572008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,8]
	 [[{{node inputs}}]]
2023-03-27 16:43:46.581303: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,8]
	 [[{{node inputs}}]]
2023-03-27 16:43:46.586870: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,256]
	 [[{{node inputs}}]]
2023-03-

--- Starting trial: run-2
{'num_units': 256, 'num_layers': 1, 'num_heads': 4, 'num_convlayers': 2, 'aggregation': 0}
26/26 [==============================] - 1s 18ms/step - loss: 1.2352 - output_1_loss: 0.5812 - output_2_loss: 0.6541 - output_1_precision_9: 0.0000e+00 - output_1_binary_accuracy: 0.7352 - output_2_precision_9: 0.7407 - output_2_binary_accuracy: 0.6269


...

2023-03-27 16:46:30.849972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,8]
	 [[{{node inputs}}]]
2023-03-27 16:46:30.858750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,8]
	 [[{{node inputs}}]]
2023-03-27 16:46:30.864112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,638,256]
	 [[{{node inputs}}]]
2023-03-

--- Starting trial: run-10
{'num_units': 256, 'num_layers': 1, 'num_heads': 8, 'num_convlayers': 2, 'aggregation': 0}
26/26 [==============================] - 1s 28ms/step - loss: 1.1847 - output_1_loss: 0.5741 - output_2_loss: 0.6106 - output_1_precision_17: 0.0000e+00 - output_1_binary_accuracy: 0.7352 - output_2_precision_17: 0.7484 - output_2_binary_accuracy: 0.7177


2023-03-27 16:46:53.168123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,636,16]
	 [[{{node inputs}}]]
2023-03-27 16:46:53.176736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,636,16]
	 [[{{node inputs}}]]
2023-03-27 16:46:53.181926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,636,256]
	 [[{{node inputs}}]]
2023-0

--- Starting trial: run-11
{'num_units': 256, 'num_layers': 1, 'num_heads': 8, 'num_convlayers': 2, 'aggregation': 1}
26/26 [==============================] - 1s 31ms/step - loss: 20.6758 - output_1_loss: 11.2120 - output_2_loss: 9.4639 - output_1_precision_18: 0.2648 - output_1_binary_accuracy: 0.2648 - output_2_precision_18: 0.3794 - output_2_binary_accuracy: 0.3794


KeyboardInterrupt: 

## Visualisation & Upload of Results

In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
!tensorboard dev upload \
  --logdir /content/logs/hparam_tuning \
  --name "Artifact Detection 6" \
  --description "Second Try Hyperparameter Optimization with models as hyperparameter - model cannot be selected in hparams" \
  --one_shot

# Random Shit

In [ ]:
                            
x_example = x_data[0:1]
y_example = y_data[0:1]

# Get the model's prediction
y_pred = model.predict(x_example)

# Calculate the binary cross-entropy loss for the example
loss_fn = weighted_binary_crossentropy(50,1)
loss_example = tf.reduce_mean(loss_fn(y_example, y_pred))

# Print the output, target values, and loss for the example
print("Model Input:\n", x_example)
print("Model Output:\n", y_pred)
print("\nTarget Values:\n", y_example)
print("\nBinary Cross-Entropy Loss for the Example:\n", loss_example)

In [ ]:
import tensorflow as tf

# Create a sample tensor of shape (9, 640)
tensor = tf.random.uniform((42, 640))

# Perform the operation: 1 - (1 - p1) * (1 - p2) * ...
result = 1 - tf.reduce_prod(1 - tensor, axis=0)

# Check the shape of the result tensor
print("Result tensor shape:", result.shape)
